In [21]:
from progressbar import Bar, ETA, \
    AdaptiveETA, Percentage, \
    ProgressBar 
widgets = [Percentage(),
            ' ', Bar(),
            ' ', ETA(),
            ' ', AdaptiveETA()]
pbar = ProgressBar(widgets=widgets)
from pcapreader import pcapToDf
import os
import pandas as pd
from tqdm.notebook import tqdm
from tabulate import tabulate
# Running cells in parallel
import ipyparallel as ipp
# run:
# ipcontroller --reuse 
# in the terminal first to start the cluster

# async def startCluster():
#     rc = await ipp.Cluster().start_and_connect(n=8, activate=True)
# await startCluster()



In [2]:
def refineLatexTableString(latexTableString):
    firstStart = latexTableString.find('{')
    formatStart = latexTableString.find('{',firstStart+1)
    firstEnd = latexTableString.find('}')
    formatEnd = latexTableString.find('}',firstEnd+1)
    replaceString = ''.join('|{}|'.format(x) for x in latexTableString[formatStart+1:formatEnd]).replace('||','|')
    latexTableString = latexTableString.replace(latexTableString[formatStart+1:formatEnd],replaceString,len(replaceString))
    newLineCounts = latexTableString.count("\n") - 1
    latexTableString = latexTableString.replace('\n','\n \hline ',newLineCounts)
    return latexTableString

In [3]:
## Get All pcap filenames:
victimFilenames = []
attackerFilenames = []
reflectorFilenames = []
for root, dirs, files in os.walk("./"):
    for file in files:
        if file.endswith(".pcapng"):
            if "victim" in file:
                victimFilenames.append(os.path.join(root, file))
            if "attacker" in file:
                attackerFilenames.append(os.path.join(root, file))
            if "reflector" in file:
                reflectorFilenames.append(os.path.join(root,file))

In [60]:
## Get attacker bytes sent
attackerBytes = []
for attackFile in pbar(attackerFilenames):
    # Get attack level
    start = attackFile.find('level')
    end = start+len('level')+1
    level = attackFile[start:end]
    attackerDf = pcapToDf(attackFile)
    # By finding the TFTP packets and summing the byte lengths we get the total number of bytes send by the attacker
    attackerDf = attackerDf.loc[attackerDf['Protocol'].isin(["TFTP"])]
    attackerBytesSent = attackerDf["Length"].sum()
    attackerBytes.append({'Level':level,'Attacker Bytes Sent':attackerBytesSent})
attackerBytes = pd.DataFrame(attackerBytes).sort_values('Level')

|████████████████████████████████████████| 6/6 [100%] in 13:26.5 (0.01/s) 


In [22]:
## Get Victim bytes received
victimBytes = []
for victimFile in pbar(victimFilenames):
    # Get attack level
    start = victimFile.find('level')
    end = start+len('level')+1
    level = victimFile[start:end]
    victimDf = pcapToDf(victimFile)
    # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
    victimBytesReceived = victimDf.loc[(victimDf['Protocol'] == "UDP") & (victimDf['UDP Destination Port'] == 50040)]["Length"].sum()
    victimBytes.append({'Level':level,'Victim Bytes Received':victimBytesReceived}) 
victimBytes = pd.DataFrame(victimBytes).sort_values('Level')

100% |############################################| Time: 0:00:10 Time: 0:00:10


In [ ]:
## Get Reflector bytes received and sent
reflectorBytes = []
for reflectorFile in pbar(reflectorFilenames):
    # Get attack level
    start = reflectorFile.find('level')
    end = start+len('level')+1
    level = reflectorFile[start:end]
    reflectorDf = pcapToDf(reflectorFile)
    # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
    reflectorBytesReceived = reflectorDf.loc[(reflectorDf['Protocol'] == "TFTP") & (reflectorDf['UDP Source Port'] == 50040)]["Length"].sum()
    reflectorBytesSent = reflectorDf.loc[(reflectorDf['Protocol'] == "TFTP") & (reflectorDf['UDP Destination Port'] == 50040)]["Length"].sum()
    reflectorBytes.append({'Level':level,'Reflector Bytes Received':reflectorBytesReceived,'Reflector Bytes Sent':reflectorBytesSent},ignore_index=True)
victreflectorBytesimBytes = pd.DataFrame(reflectorBytes).sort_values('Level')

In [ ]:
## Data manipulation for presentation

extracted_col = attackerBytes['Attacker Bytes Sent']
byteStatisticTable = victimBytes.join(extracted_col)
byteStatisticTable['Amplification factor'] = byteStatisticTable['Victim Bytes Received']/byteStatisticTable['Attacker Bytes Sent']

In [9]:
print(tabulate(byteStatisticTable,headers='keys',tablefmt='fancy_grid',showindex=False))
latexTableString = byteStatisticTable.style.to_latex() # For getting the table into the report
latexTableString = refineLatexTableString(latexTableString)

print(latexTableString)

╒═════════╤═════════════════════════╤═══════════════════════╤════════════════════════╕
│ Level   │   Victim Bytes Received │   Attacker Bytes Sent │   Amplification factor │
╞═════════╪═════════════════════════╪═══════════════════════╪════════════════════════╡
│ level0  │                   48104 │                   840 │             57.2667    │
├─────────┼─────────────────────────┼───────────────────────┼────────────────────────┤
│ level1  │                  508528 │                  8880 │             57.2667    │
├─────────┼─────────────────────────┼───────────────────────┼────────────────────────┤
│ level2  │                 1813712 │                 80640 │             22.4915    │
├─────────┼─────────────────────────┼───────────────────────┼────────────────────────┤
│ level3  │                 2701042 │                781080 │              3.45809   │
├─────────┼─────────────────────────┼───────────────────────┼────────────────────────┤
│ level4  │                 3171468 │      